<a href="https://colab.research.google.com/github/Ayman-Abdeen/DataAnalysis-A/blob/main/Regression_Ensemble_Methods_Bagging%2CBoosting%2CStacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
df = pd.read_excel("Real_Estate_data.xlsx",'Real_Estate_data')
print (df.count())
SVMdf = df.head(2000)

SalePrice      5891
BuildingAge    5891
Area           5891
Floor          5891
Location       5891
dtype: int64


In [8]:
df.head()

,SalePrice,BuildingAge,Area,Floor,Location
0,141592,2006,814,3,1
1,51327,1985,587,8,1
2,48672,1985,587,6,1
3,380530,2006,2056,8,0
4,221238,1993,1761,3,0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5891 entries, 0 to 5890
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   SalePrice    5891 non-null   int64
 1   BuildingAge  5891 non-null   int64
 2   Area         5891 non-null   int64
 3   Floor        5891 non-null   int64
 4   Location     5891 non-null   int64
dtypes: int64(5)
memory usage: 230.2 KB


In [12]:
df.describe()

,SalePrice,BuildingAge,Area,Floor,Location
count,5891.000000,5891.000000,5891.000000,5891.000000,5891.000000
mean,221218.112545,2002.967408,955.569173,12.026311,1.111866
std,106384.186446,8.811782,382.464050,7.548743,0.351393
min,32743.000000,1978.000000,135.000000,1.000000,0.000000
25%,144247.000000,1993.000000,644.000000,6.000000,1.000000
50%,207964.000000,2006.000000,910.000000,11.000000,1.000000
75%,291150.000000,2008.000000,1149.000000,17.000000,1.000000
max,585840.000000,2015.000000,2337.000000,43.000000,2.000000


In [3]:
df.corr() #get the correlation matrix to see to which level the dataset attributes are correlated, that will affect selecting attributes in building the forecasting model
# the matrix inform us that there is a strong relationship between pricem age, area, and floor
#df.corr()['SalePrice'] #to get correlation for specific field with other atributes

,SalePrice,BuildingAge,Area,Floor,Location
SalePrice,1.000000,0.447875,0.697199,0.336729,0.010615
BuildingAge,0.447875,1.000000,0.021282,0.260568,0.005180
Area,0.697199,0.021282,1.000000,0.179491,0.017981
Floor,0.336729,0.260568,0.179491,1.000000,0.005355
Location,0.010615,0.005180,0.017981,0.005355,1.000000


In [5]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
#selection to attributes where according to the correlation
Xtest = df[['BuildingAge','Area','Floor','Location']] 
X = df[['BuildingAge','Area','Floor']]
y = df[['SalePrice']]

from sklearn.feature_selection import RFE

rfe = RFE(regressor,n_features_to_select=3,step=43) # you can tell the eliminator how many features you need
fit = rfe.fit(Xtest, y) # let him calculate it free way
print("Num Features: %d" % fit.n_features_)
print("Selected Features: %s" % fit.support_) #False means never to exclude it
print("Feature Ranking: %s" % fit.ranking_) #the more value the more will impact your model

for i in range(Xtest.shape[1]):
	print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))

Num Features: 3
Selected Features: [ True False  True  True]
Feature Ranking: [1 2 1 1]
Column: 0, Selected True, Rank: 1.000
Column: 1, Selected False, Rank: 2.000
Column: 2, Selected True, Rank: 1.000
Column: 3, Selected True, Rank: 1.000


In [6]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
chiSelector = SelectKBest(chi2,k=3)
X_kbest = chiSelector.fit(Xtest, y)
print(X_kbest.scores_)#We choose the highest values, which will be the first 3,selects the features according to the k highest score

[1.18453147e+02 5.61716913e+05 9.62733118e+03 1.10589874e+02]


In [7]:
X = df[['BuildingAge','Area','Floor']]
y = df[['SalePrice']]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75)

Baseline the linear regression to compare with

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7)

LRRegressor = LinearRegression() #gives less accuracy R^2: 0.6826995509387148

LRRegressor.fit(X_train, y_train)
print (LRRegressor.predict([[1995,280,10]]))#prediction for apartment, with building age 1995, area 280, and floor 10#Model Evaluation
from sklearn import metrics
y_pred = LRRegressor.predict(X_test)
#Model Evaluation
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_pred))
#print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_pred))
print('Explained Variance Score:', metrics.explained_variance_score(y_test, y_pred))
#print('Mean Squared Log Error:', metrics.mean_squared_log_error(y_test, y_pred))
print('Median Absolute Error:', metrics.median_absolute_error(y_test, y_pred))
print('R^2:', metrics.r2_score(y_test, y_pred)) #Check the accuracty when changing the sample size, when getting negative value, 
#R2 is negative only when the chosen model does not follow the trend of the data. It seems that your model may be giving better performance because of over-fitting. It can be a case of over-fitting in the model. It can happen because of various reasons like small dataset and noise in the dataset.Jun 13, 2021


[[57942.09109202]]
Mean Absolute Error (MAE): 48531.13952696794
Explained Variance Score: 0.6926044803510806
Median Absolute Error: 43967.26967455633
R^2: 0.6924217009555542


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  "X does not have valid feature names, but"


Bagging Methods : Random Forest

In [14]:
#ensemble regression
from sklearn.ensemble import RandomForestRegressor  #bagging technique
from sklearn import metrics

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=0)
RFregressor = RandomForestRegressor(n_estimators = 10)
RFregressor.fit(X_train, y_train)
print (RFregressor.predict([[1995,280,10]]))#prediction for apartment, with building age 1995, area 280, and floor 10#Model Evaluation
y_pred = RFregressor.predict(X_test)
#Model Evaluation
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_pred))
print('Explained Variance Score:', metrics.explained_variance_score(y_test, y_pred))
print('Mean Squared Log Error:', metrics.mean_squared_log_error(y_test, y_pred))
print('Median Absolute Error:', metrics.median_absolute_error(y_test, y_pred))
print('R^2:', metrics.r2_score(y_test, y_pred)) #Check the accuracty when changing the sample size, when getting negative value, 
#R2 is negative only when the chosen model does not follow the trend of the data. It seems that your model may be giving better performance because of over-fitting. It can be a case of over-fitting in the model. It can happen because of various reasons like small dataset and noise in the dataset.Jun 13, 2021


[73825.85714286]
Mean Absolute Error (MAE): 38486.942620909926
Mean Squared Error (MSE): 2543471407.612492
Explained Variance Score: 0.7755624713541796
Mean Squared Log Error: 0.06909065644256912
Median Absolute Error: 30973.79999999999
R^2: 0.7753044774211497


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  "X does not have valid feature names, but"


In [15]:
#ensemble regression
from sklearn.ensemble import BaggingRegressor  #bagging technique
from sklearn import metrics

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=0)
baggingregressor = BaggingRegressor(n_estimators = 10) #estimators means no. trees , max depth of the tree
baggingregressor.fit(X_train, y_train)
print (baggingregressor.predict([[1995,280,10]]))#prediction for apartment, with building age 1995, area 280, and floor 10#Model Evaluation
y_pred = baggingregressor.predict(X_test)
#Model Evaluation
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_pred))
print('Explained Variance Score:', metrics.explained_variance_score(y_test, y_pred))
print('Mean Squared Log Error:', metrics.mean_squared_log_error(y_test, y_pred))
print('Median Absolute Error:', metrics.median_absolute_error(y_test, y_pred))
print('R^2:', metrics.r2_score(y_test, y_pred)) #Check the accuracty when changing the sample size, when getting negative value, 
#R2 is negative only when the chosen model does not follow the trend of the data. It seems that your model may be giving better performance because of over-fitting. It can be a case of over-fitting in the model. It can happen because of various reasons like small dataset and noise in the dataset.Jun 13, 2021


[76435.43809524]
Mean Absolute Error (MAE): 38566.53283075596
Mean Squared Error (MSE): 2574998758.541712
Explained Variance Score: 0.7727352553847979
Mean Squared Log Error: 0.06956602767985945
Median Absolute Error: 31090.173333333325
R^2: 0.7725192860597034


/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:429: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but BaggingRegressor was fitted with feature names
  "X does not have valid feature names, but"


Boosting Methods : Gradient Boosting

In [16]:
#ensemble regression
from sklearn.ensemble import GradientBoostingRegressor # boosting technique
from sklearn import metrics

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.65, random_state=0)
GBregressor = GradientBoostingRegressor()
GBregressor.fit(X_train, y_train)
print (GBregressor.predict([[1995,280,10]]))#prediction for apartment, with building age 1995, area 280, and floor 10#Model Evaluation
y_pred = GBregressor.predict(X_test)
#Model Evaluation
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_pred))
print('Explained Variance Score:', metrics.explained_variance_score(y_test, y_pred))
print('Mean Squared Log Error:', metrics.mean_squared_log_error(y_test, y_pred))
print('Median Absolute Error:', metrics.median_absolute_error(y_test, y_pred))
print('R^2:', metrics.r2_score(y_test, y_pred)) #Check the accuracty when changing the sample size, when getting negative value, 
#R2 is negative only when the chosen model does not follow the trend of the data. It seems that your model may be giving better performance because of over-fitting. It can be a case of over-fitting in the model. It can happen because of various reasons like small dataset and noise in the dataset.Jun 13, 2021


[58822.56109666]
Mean Absolute Error (MAE): 35506.470869020224
Mean Squared Error (MSE): 1967472719.543909
Explained Variance Score: 0.825422239537838
Mean Squared Log Error: 0.053889399171574114
Median Absolute Error: 29933.126897092763
R^2: 0.8253199316808023


/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  "X does not have valid feature names, but"


In [17]:
from sklearn.ensemble import StackingRegressor #Stacking regressor
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeRegressor 
from sklearn.svm import LinearSVR 

import warnings
warnings.filterwarnings("ignore")

estimators = [('Decision tree', DecisionTreeRegressor()),
('Random Forest', RandomForestRegressor()),
('SVR', LinearSVR(random_state=42))              
             ]

stackingreg = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
stackingreg.fit(X_train, y_train).score(X_test, y_test)

score = cross_val_score(stackingreg,X,y,cv = 15,scoring = 'r2')
print("The accuracy score of is:",score.mean())
print (stackingreg.predict([[1995,280,10]]))#prediction for apartment, with building age 1995, area 280, and floor 10#Model Evaluation
y_pred = stackingreg.predict(X_test)
#Model Evaluation
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_pred))
print('Explained Variance Score:', metrics.explained_variance_score(y_test, y_pred))
print('Mean Squared Log Error:', metrics.mean_squared_log_error(y_test, y_pred))
print('Median Absolute Error:', metrics.median_absolute_error(y_test, y_pred))
print('R^2:', metrics.r2_score(y_test, y_pred)) #Check the accuracty when changing the sample size, when getting negative value, 
#R2 is negative only when the chosen model does not follow the trend of the data. It seems that your model may be giving better performance because of over-fitting. It can be a case of over-fitting in the model. It can happen because of various reasons like small dataset and noise in the dataset.Jun 13, 2021


The accuracy score of is: 0.724947140088991
[77222.74164696]
Mean Absolute Error (MAE): 37873.56840806454
Mean Squared Error (MSE): 2259491200.659704
Explained Variance Score: 0.7942152462115566
Mean Squared Log Error: 0.06198201373797744
Median Absolute Error: 32919.33856715771
R^2: 0.7942146331408386
